## Importação de Biblioteca

In [ ]:
!pip install meteostat

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
import seaborn as sns
# from imblearn.over_sampling import SMOTE
import gdown
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
arquivo_destino_base = "dataset_{}.csv"

ids = {
    "consumo_2024": "1-iXT7eaJWQokHf9cyfrB8N5wvkdhgjJW",
    "consumo_2023": "1-WfvkRwaRr85B_Joxcm9xVdpyg5NBAmp",
    "consumo_2022": "1-Uu4Tf4lufJVFeJnYKc5w7OeW66pe1eC",
    "consumo_2021": "1-2PsTLzG4dcY4wM0p7vFfabUuLv950gC",
    # "consumo_2020": "1-1pOoa0eJlNJ94BMi7p4PTx5KUS96mhX",
    "consumo_2019": "1-2PsTLzG4dcY4wM0p7vFfabUuLv950gC"
}


dataframes = {}


for key, file_id in ids.items():
    url = f"https://drive.google.com/uc?id={file_id}"
    arquivo_destino = arquivo_destino_base.format(key)

    gdown.download(url, arquivo_destino, quiet=False)

In [ ]:
# juntando todos os dataframes em um só

arquivos_csv = [
    "./dataset_consumo_2024.csv",
    "./dataset_consumo_2023.csv",
    "./dataset_consumo_2022.csv",
    "./dataset_consumo_2021.csv",
    "./dataset_consumo_2019.csv"
]

ALL_COLUMNS_CONSUMO_GERAL = pd.concat([pd.read_csv(arquivo, delimiter=";") for arquivo in arquivos_csv], axis=0)

In [ ]:
ALL_COLUMNS_CONSUMO_GERAL['REFERENCIA']

In [ ]:
def cortandoData(df, startAno, startMes, endAno, endMes):
    start_date = f'{startAno}-{str(startMes).zfill(2)}-01'
    end_date = f'{endAno}-{str(endMes).zfill(2)}-01'

    df['REFERENCIA'] = pd.to_datetime(df['REFERENCIA'], format='%Y-%m-%d')

    # Filtrando o DataFrame pela janela de tempo
    mask = (df['REFERENCIA'] >= start_date) & (df['REFERENCIA'] <= end_date)
    df_filtered = df.loc[mask]

    return df_filtered

ALL_COLUMNS_CONSUMO_GERAL = cortandoData(ALL_COLUMNS_CONSUMO_GERAL, 2022, 1, 2024, 1)

In [ ]:
ALL_COLUMNS_CONSUMO_GERAL['REFERENCIA'].unique()

In [ ]:
file_id_fraudes = "1-MbIlChqQapcxFkoJgpbQIsN9FBLfbX1"
url_fraudes = f"https://drive.google.com/uc?id={file_id_fraudes}"

gdown.download(url_fraudes, quiet=False)

# LEMBRAR DE TENTAR ADICIONAR ECO_RESIDENCIAL

## Temperatura

In [ ]:
import plotly.graph_objects as go
from meteostat import Point, Daily
from datetime import datetime

# Define a localização geográfica: Campo Grande MT
location = Point(-20.4435, -54.6478)

# Define o intervalo de datas
start = datetime(2022, 1, 1)
end = datetime(2024, 1, 1)

# Busca os dados diários de clima para a localização e período definidos
data = Daily(location, start, end)
data = data.fetch()

# Calcula a média mensal dos dados climáticos
df_monthly_weather_23 = data.resample('M').mean()

# Renomeia as colunas do DataFrame para termos mais descritivos em português
df_weather_23 = df_monthly_weather_23.rename(columns={
    'tavg': 'Temperatura Média',
    'tmin': 'Temperatura Mínima',
    'tmax': 'Temperatura Máxima',
    'prcp': 'Precipitação',
    'snow': 'Neve',
    'wdir': 'Direção do Vento',
    'wspd': 'Velocidade do Vento',
    'wpgt': 'Rajada Máxima de Vento',
    'pres': 'Pressão Atmosférica',
    'tsun': 'Insolação'
})

In [ ]:
df_weather_23 = df_weather_23.drop(['Neve',	'Direção do Vento', 'Velocidade do Vento',	'Rajada Máxima de Vento', 'Pressão Atmosférica',	'Insolação'], axis=1)
df_weather_23

In [ ]:
len(df_weather_23)

In [ ]:
ALL_COLUMNS_CONSUMO_GERAL['REFERENCIA'].value_counts()

In [ ]:
ALL_COLUMNS_CONSUMO_GERAL.columns.values

In [ ]:
ALL_COLUMNS_CONSUMO_GERAL['REFERENCIA'] = pd.to_datetime(ALL_COLUMNS_CONSUMO_GERAL['REFERENCIA'])

ALL_COLUMNS_CONSUMO_GERAL['MES'] = ALL_COLUMNS_CONSUMO_GERAL['REFERENCIA'].dt.month

df_weather_23['MES'] = df_weather_23.index.month

# Agrupa os dados por mês e calcula a média do consumo medido ('CONS_MEDIDO') para cada mês
consumo_medio_23 = ALL_COLUMNS_CONSUMO_GERAL.groupby('MES')['CONS_MEDIDO'].mean().reset_index()

# Combina os dados de consumo médio com os dados meteorológicos com base na coluna 'MES'
ALL_COLUMNS_CONSUMO_GERAL = pd.merge(ALL_COLUMNS_CONSUMO_GERAL, df_weather_23, on='MES')

In [ ]:
ALL_COLUMNS_CONSUMO_GERAL

In [ ]:
# # diminuir dataset, reduzindo o numero de matrículas

# def filtrar_por_matriculas(df, n=1000):
#     matriculas = df['MATRICULA'].unique()[:n]

#     df_filtrado = df[df['MATRICULA'].isin(matriculas)]

#     return df_filtrado


# df = filtrar_por_matriculas(ALL_COLUMNS_CONSUMO_GERAL)

In [ ]:
fraudes = pd.read_csv('/content/fraudes.csv')

In [ ]:
fraudes.columns.values

In [ ]:
def cortandoDataFraude(df, startAno, endAno):
    start_date = f'{startAno}'
    end_date = f'{endAno}'

    df['ANOOS'] = pd.to_datetime(df['ANOOS'], format='%Y')

    # Filtrando o DataFrame pela janela de tempo
    mask = (df['ANOOS'] >= start_date) & (df['ANOOS'] <= end_date)
    df_filtered = df.loc[mask]

    return df_filtered

fraudes = cortandoDataFraude(fraudes, 2022, 2024)

In [ ]:
# removendo colunas que não seram utilizandas

ALL_COLUMNS_CONSUMO_GERAL = ALL_COLUMNS_CONSUMO_GERAL.drop(columns=['Unnamed: 0', "SUB_CATEGORIA", "STA_TROCA", "STA_ACEITA_LEITURA", 'STA_TROCA', 'EMP_CODIGO', 'COD_GRUPO', 'COD_SETOR_COMERCIAL', 'NUM_QUADRA', 'COD_ROTA_LEITURA', 'SEQ_RESPONSAVEL', 'ECO_RESIDENCIAL', 'ECO_COMERCIAL', 'ECO_INDUSTRIAL', 'ECO_PUBLICA', 'ECO_OUTRAS','LTR_ATUAL', 'LTR_COLETADA', 'DIAS_LEITURA', 'COD_LEITURA_INF_1', 'COD_LEITURA_INF_2', 'COD_LEITURA_INF_3', 'HORA_LEITURA', 'DSC_SIMULTANEA', 'COD_LEITURA_INT','EXCECAO'])

In [ ]:
ALL_COLUMNS_CONSUMO_GERAL.head()

In [ ]:
ALL_COLUMNS_CONSUMO_GERAL['MATRICULA'].dtype

In [ ]:
ALL_COLUMNS_CONSUMO_GERAL_UMA_MATRICULA = ALL_COLUMNS_CONSUMO_GERAL[ALL_COLUMNS_CONSUMO_GERAL['MATRICULA'] == 17719490.0]
ALL_COLUMNS_CONSUMO_GERAL_UMA_MATRICULA['REFERENCIA'] = pd.to_datetime(ALL_COLUMNS_CONSUMO_GERAL_UMA_MATRICULA['REFERENCIA'])

# Agrupando o consumo por data de referência
df_agrupado = ALL_COLUMNS_CONSUMO_GERAL_UMA_MATRICULA.groupby('REFERENCIA')['CONS_MEDIDO'].sum().reset_index()

# Plotando a distribuição do consumo medido por data
# Plotando a distribuição do consumo medido por data utilizando gráfico de área
plt.figure(figsize=(10, 6))
plt.fill_between(df_agrupado['REFERENCIA'], df_agrupado['CONS_MEDIDO'], color='blue', alpha=0.5)
plt.title('Distribuição do Consumo Medido por Data de Referência')
plt.xlabel('Data de Referência')
plt.ylabel('Consumo Medido (m³)')
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()

In [ ]:
import plotly.express as px

fig = px.line(df_agrupado, x='REFERENCIA', y='CONS_MEDIDO',
              title='Distribuição do Consumo Medido por Data de Referência - Matrícula 17719490',
              labels={'REFERENCIA': 'Data de Referência', 'CONS_MEDIDO': 'Consumo Medido (m³)'})

# Ajustando o layout do gráfico
fig.update_layout(xaxis_title='Data de Referência',
                  yaxis_title='Consumo Medido (m³)',
                  xaxis_tickangle=-45)

# Exibir o gráfico
fig.show()

In [ ]:
# # remove linhas duplicadas

# def verificar_duplicatas_referencia(df):
#     df_duplicados = df[df.duplicated(subset=['MATRICULA', 'REFERENCIA'], keep=False)]

#     return df_duplicados

# df_sem_duplicadas = verificar_duplicatas_referencia(ALL_COLUMNS_CONSUMO_GERAL)

In [ ]:
sem_na = ALL_COLUMNS_CONSUMO_GERAL.dropna()

In [ ]:
sem_na

In [ ]:
def remove_rows_with_column_value_greater_than_one(df, column_names):
    # Verifica se algum valor em qualquer uma das colunas especificadas é maior que 1
    condition = (df[column_names] > 1).any(axis=1)

    # Filtra o DataFrame, removendo as linhas onde a condição é True
    df_filtered = df[~condition]

    return df_filtered

# Exemplo de uso:
df_sem_linhas = remove_rows_with_column_value_greater_than_one(sem_na, ['COD_LATITUDE', 'COD_LONGITUDE'])

In [ ]:
# tratando a longetude e latitude
# verificando a quantidade de clusters é mais apropriada utilizando K-Means


def plotar(n_clusters):
  #clusterizando
  df_temp = df_sem_linhas.copy()
  kmeans = KMeans(n_clusters=n_clusters, random_state=42)
  df_temp['cluster'] = kmeans.fit_predict(df_sem_linhas[['COD_LATITUDE', 'COD_LONGITUDE']])

  # visualizando
  plt.figure(figsize=(10, 6))
  plt.scatter(df_sem_linhas['COD_LONGITUDE'], df_sem_linhas['COD_LATITUDE'], c=df_temp['cluster'], cmap='viridis', marker='o', s=100)
  plt.title(f"K-Means Clustering com {n_clusters} Clusters")
  plt.xlabel("Longitude")
  plt.ylabel("Latitude")
  plt.show()



# visulizar clusters do range de 2 para 17
# for x in range(2, 17):
#   print()
#   plotar(x)

In [ ]:
# clusterizando os dados de longetude e latitude

kmeans = KMeans(n_clusters=20, random_state=42)
df_sem_linhas['cluster'] = kmeans.fit_predict(df_sem_linhas[['COD_LATITUDE', 'COD_LONGITUDE']])

In [ ]:
# removendo colunas de longetude e latude

df_loc_tratado = df_sem_linhas.drop(columns=['COD_LATITUDE', 'COD_LONGITUDE'])

In [ ]:
# fazendo one hot encoding

df_loc_tratado_ohc = pd.get_dummies(df_loc_tratado, columns = ['cluster'], dtype=int)

In [ ]:
df_loc_tratado_ohc.columns

In [ ]:
df_loc_tratado_ohc = df_loc_tratado_ohc[df_loc_tratado_ohc["CATEGORIA"].isin(["RESIDENCIAL"])]
df_loc_tratado_ohc = df_loc_tratado_ohc[df_loc_tratado_ohc["CATEGORIA"].isin(["RESIDENCIAL"])]
df_loc_tratado_ohc = df_loc_tratado_ohc.drop(columns="CATEGORIA")

In [ ]:
scaler = RobustScaler()
df_loc_tratado_ohc[['VOLUME_ESTIMADO_ACUM', 'VOLUME_ESTIMADO']] = scaler.fit_transform(df_loc_tratado_ohc[['VOLUME_ESTIMADO_ACUM', 'VOLUME_ESTIMADO']])

In [ ]:
df_loc_tratado_ohc['Temperatura Média'] = df_loc_tratado_ohc['Temperatura Média'].astype(int)
df_loc_tratado_ohc['Temperatura Mínima'] = df_loc_tratado_ohc['Temperatura Mínima'].astype(int)
df_loc_tratado_ohc['Temperatura Máxima'] = df_loc_tratado_ohc['Temperatura Máxima'].astype(int)
df_loc_tratado_ohc['Precipitação'] = df_loc_tratado_ohc['Precipitação'].astype(int)

In [ ]:
df_loc_tratado_ohc['Precipitação'].isna().sum()

In [ ]:
# pivotando a tabela

pivoted_df = pd.pivot_table(
    df_loc_tratado_ohc,
    index='MATRICULA',
    columns='REFERENCIA',
    values=['CONS_MEDIDO', 'VOLUME_ESTIMADO', 'Temperatura Média', 'Precipitação'],
    aggfunc='sum'
)

pivoted_df.columns = ['_'.join([str(col[0]), col[1].strftime('%Y-%m-%d')]).strip() for col in pivoted_df.columns.values]
pivoted_df = pivoted_df.reset_index()


# adicinando as outras colunas

other_columns = df_loc_tratado_ohc.drop(columns=['CONS_MEDIDO', 'VOLUME_ESTIMADO', 'REFERENCIA']).drop_duplicates()


pivoted_df = pd.merge(pivoted_df, other_columns, on='MATRICULA', how='left')


pivoted_df = pivoted_df.fillna(0)

In [ ]:
pivoted_df.columns.values

In [ ]:
pivoted_df = pivoted_df[pivoted_df['DSC_OCORRENCIA'].isin([
    'NORMAL',
    'MEDIDOR RETIRADO/FURTADO',
    'MEDIDOR INVERTIDO/LIGAÇÃO LACRADA',
    'LEITURA COLETADA PELO CLIENTE',
    'MEDIDOR NÃO LOCALIZADO',
    'IMÓVEL DESOCUPADO',
    'HIDRÔMETRO COM CÚPULA VIRADA/TAMPADA',
    'CAIXA DE PROTEÇÃO FECHADA',
    'IMÓVEL A ALUGAR/VENDA'
    'LIGAÇÃO NÃO LOCALIZADA'
    'FRAUDE IDENTIFICADA'
    'MEDIDOR COM DEFEITO/PARADO'
    'CACHORRO BRAVO'
])]

In [ ]:
pivoted_df["DSC_OCORRENCIA"].value_counts()

In [ ]:
# balanciando e simplificando a coluna do tipo de ocorrencia para normal e não normal


result = pivoted_df.copy()

result['DSC_OCORRENCIA'] = pivoted_df['DSC_OCORRENCIA'].apply(lambda x: 1 if x == 'NORMAL' else 0)

result.head()

In [ ]:
result = result.drop(columns=['FATURADO_MEDIA'])

In [ ]:
result["TIPO_LIGACAO"] = result["TIPO_LIGACAO"].replace({"Hidrometrado": 0, "Consumo Fixo": 1})

In [ ]:
result["TIPO_LIGACAO"] = result["TIPO_LIGACAO"].astype(int)

In [ ]:
result["TIPO_LIGACAO"].value_counts()

In [ ]:
result.dtypes

In [ ]:
def remove_outliers_and_plot(df, column_name):
    # 1. Calcular os quartis
    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)

    # 2. Calcular o IQR
    IQR = Q3 - Q1

    # 3. Determinar os limites superior e inferior
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    # 4. Filtrar os dados para remover os outliers
    df_filtrado = df[(df[column_name] >= limite_inferior) & (df[column_name] <= limite_superior)]

    # 5. Plotar o boxplot com os dados filtrados
    sns.boxplot(x=df_filtrado[column_name])
    plt.title(f'Boxplot de {column_name} (Sem Outliers)')
    plt.xlabel(column_name)
    plt.show()

    # Retornar o DataFrame filtrado
    return df_filtrado

# Exemplo de uso:
df_sem_outliers = remove_outliers_and_plot(result, 'VOLUME_ESTIMADO_ACUM')

In [ ]:
df_sem_outliers["MATRICULA"] = df_sem_outliers["MATRICULA"].astype(int)

In [ ]:
df_sem_outliers

In [ ]:
dataframe_fraudes_premissa = fraudes[['MATRICULA', 'DESCRICAO']].drop_duplicates()
dataframe_fraudes_premissa = pd.get_dummies(dataframe_fraudes_premissa, columns=['DESCRICAO'], dtype='int')

In [ ]:
dataframe_pj_premissa = pd.merge(df_sem_outliers, dataframe_fraudes_premissa, on='MATRICULA', how='left')

In [ ]:
dataframe_pj_premissa['DESCRICAO_IRREGULARIDADE IDENTIFICADA'] = dataframe_pj_premissa['DESCRICAO_IRREGULARIDADE IDENTIFICADA'].fillna(0)

In [ ]:
dataframe_pj_premissa['DESCRICAO_IRREGULARIDADE IDENTIFICADA'].value_counts()

In [ ]:
dataframe_pj_premissa['DESCRICAO_IRREGULARIDADE IDENTIFICADA'] = dataframe_pj_premissa['DESCRICAO_IRREGULARIDADE IDENTIFICADA'].astype(int)

In [ ]:
# def balanciar(df):
#   smote = SMOTE(random_state=42)
#   X = df.drop(['DESCRICAO_IRREGULARIDADE IDENTIFICADA', 'REFERENCIA'], axis=1)
#   y = df['DESCRICAO_IRREGULARIDADE IDENTIFICADA']
#   X_res, y_res = smote.fit_resample(X, y)
#   return pd.concat([X_res, y_res], axis=1)


# dataframe_pj_premissa = balanciar(dataframe_pj_premissa)

In [ ]:
# rus = RandomUnderSampler(random_state=42)
# X = pivoted_df.drop('DESCRICAO_IRREGULARIDADE IDENTIFICADA', axis=1)
# y = pivoted_df['DESCRICAO_IRREGULARIDADE IDENTIFICADA']
# X_res, y_res = rus.fit_resample(X, y)

In [ ]:
np.unique(dataframe_pj_premissa['DESCRICAO_IRREGULARIDADE IDENTIFICADA'].values, return_counts=True)

In [ ]:
dataframe_pj_premissa.columns.values

In [ ]:
dataframe_pj_premissa.drop(columns=['Temperatura Média', 'Temperatura Mínima', 'Temperatura Máxima','Precipitação'], inplace=True)

In [ ]:
dataframe_pj_premissa

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dataframe_pj_premissa.to_parquet('/content/drive/Shareddrives/LeakSeeker/pre_processado_pf.parquet')

In [ ]:
# import folium
# from sklearn.cluster import KMeans

# centroides = kmeans.cluster_centers_

# # Criar um mapa centrado nas coordenadas médias do dataset
# media_lat = df_sem_linhas['COD_LATITUDE'].mean()
# media_long = df_sem_linhas['COD_LONGITUDE'].mean()
# mapa = folium.Map(location=[media_lat, media_long], zoom_start=12)

# # Adicionar os pontos dos dados ao mapa
# for i, row in df_sem_linhas.iterrows():
#     folium.CircleMarker(location=[row['COD_LATITUDE'], row['COD_LONGITUDE']],
#                         radius=5,
#                         color='blue',
#                         fill=True,
#                         fill_color='blue',
#                         fill_opacity=0.6).add_to(mapa)

# # Adicionar os centróides dos clusters ao mapa com o número do cluster
# for i, centroide in enumerate(centroides):
#     folium.Marker(location=[centroide[0], centroide[1]],
#                   popup=f'Cluster {i}',  # Exibe o número do cluster
#                   icon=folium.DivIcon(html=f"""<div style="font-size: 12pt; color : red">{i}</div>""")).add_to(mapa)

# # Mostrar o mapa
# mapa

### Comportamento das Matrículas

In [ ]:
fraudulentos = dataframe_pj_premissa[dataframe_pj_premissa['DESCRICAO_IRREGULARIDADE IDENTIFICADA'] == 1].head(5)
nao_fraudulentos = dataframe_pj_premissa[dataframe_pj_premissa['DESCRICAO_IRREGULARIDADE IDENTIFICADA'] == 0].head(5)


colunas_consumo = [col for col in dataframe_pj_premissa.columns if 'CONS_MEDIDO' in col]

# Empilhar os dados para facilitar o plot
dados_fraude = fraudulentos[colunas_consumo].stack()
dados_nao_fraude = nao_fraudulentos[colunas_consumo].stack()

# 3. Criar o histograma
plt.figure(figsize=(12,6))

sns.histplot(dados_fraude, color='red', label='Fraudulentos', kde=True, bins=10)
sns.histplot(dados_nao_fraude, color='blue', label='Não Fraudulentos', kde=True, bins=10)

plt.title('Comparação de Consumo entre Fraudulentos e Não Fraudulentos')
plt.xlabel('Consumo Medido')
plt.ylabel('Frequência')
plt.legend()
plt.show()